## Experiments

Pipeline Steps:

*from Driffta*

- Given region and tile
    - load Experiment.json, channelNames.txt, and processingOptions.json
    - for each channel + z
        - construct input image name
        - if not keyence, copy the file first before opening?
        - if channel_arrangement is "color" do max z-projection
        - \* get explanation on what to do with H&E
        - aggregate images into stack
    - choose best focus z-plane for each cycle
        - currently, the same plane is used for all cycles though
    - run drift compensation
    - crop off tile overlap 
        - note that this is different when "useBleachMinimizingCrop" is set in processingOptions
    - run deconvolution
    - set LUTS (for either 3 or 4 channels)
    - do background subtraction if set by experiment.bgSub
    - save result
    - also save single z-stack using best focus plans in separate dir
    


In [1]:
from codex import config

In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=3, processes=True)
client = Client(cluster)
client

In [5]:
client

Client Scheduler: tcp://127.0.0.1:58463,Cluster Workers: 3 Cores: 9 Memory: 19.33 GB


In [11]:
import dask.array as da

def fn(a):
    import time
    import numpy as np
    for i in range(100):
        np.matmul(np.ones((1000, 1000)), np.ones((1000, 1000)))
    return a*2
res = client.map(fn, [1, 2, 3])
res

[<Future: status: pending, key: fn-50ac6fd4db73382b2d803f919be8a238>,
 <Future: status: pending, key: fn-b98761c5920ab097086c24e37aaf27e8>,
 <Future: status: pending, key: fn-370c033fb5625485f704265759103b71>]

In [19]:
[r.result() for r in res]

[2, 4, 6]

In [21]:
cluster.close()

In [19]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10807682206669945409]

In [28]:
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
import numpy as np

def run_mult(device):
    for i in range(1000):
        np.matmul(np.ones((1000, 1000)), np.ones((1000, 1000)))
    return 1

def run_deconvolution(device):
    acq = fd_data.bars_25pct()
    algo = fd_restoration.RichardsonLucyDeconvolver(3).initialize()
    res = algo.run(acq, niter=1000).data
    return res

In [29]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=3, processes=True)
client = Client(cluster)

try:
    #res = client.map(run_deconvolution, ['/cpu:0']*3)
    res = client.map(run_mult, ['/cpu:0']*3)
    res = [r.result() for r in res]
finally:
    client.close()
    cluster.close()

In [17]:
len(res[0])

32